In [1]:
from dotenv import load_dotenv
from haystack import Pipeline
from haystack.components.builders import ChatPromptBuilder
from haystack.components.converters import TextFileToDocument
from haystack.components.embedders import SentenceTransformersDocumentEmbedder, SentenceTransformersTextEmbedder
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.dataclasses import ChatMessage
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.utils import Secret
from pathlib import Path

# Initialize document store
document_store = InMemoryDocumentStore()

# Load files and convert to documents
files = [f for f in Path("./example_data").iterdir() if f.is_file()]
converter = TextFileToDocument()
docs = converter.run(sources=files)['documents']

# Write documents to the store
doc_embedder = SentenceTransformersDocumentEmbedder(
    model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()

docs_with_embeddings = doc_embedder.run(docs)
document_store.write_documents(docs_with_embeddings["documents"])

/Users/jama/programming/iff-haystack-rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|██████████| 1/1 [00:00<00:00,  4.10it/s]


5

In [2]:
load_dotenv()
# Define the parts of the pipeline
text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
retriever = InMemoryEmbeddingRetriever(document_store)
llm = OpenAIChatGenerator(model="gpt-4o-mini", api_key=Secret.from_env_var("OPENAI_KEY"))
prompt_builder = ChatPromptBuilder(template=[
    ChatMessage.from_user("""
                          Given the following information, answer the question.

                          Context:
                          {% for doc in documents %}
                            {{ doc.content }}
                          {% endfor %}

                          Question: {{ question }}
                          Answer:
                          """)
])

# Create a pipeline
pipe = Pipeline()

pipe.add_component("text_embedder", text_embedder)
pipe.add_component("retriever", retriever)
pipe.add_component("llm", llm)
pipe.add_component("prompt_builder", prompt_builder)

pipe.connect("text_embedder.embedding", "retriever.query_embedding")
pipe.connect("retriever", "prompt_builder")
pipe.connect("prompt_builder.prompt", "llm.messages")

# Example query
question = "What kind of economic software do we use?"
response = pipe.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})
print(response['llm']['replies'][0].text)


Batches: 100%|██████████| 1/1 [00:00<00:00,  7.40it/s]


Eir uses the following financial tools for its accounting and finance operations:

1. **Visma**: This software is utilized for payroll and bookkeeping tasks.
2. **Tripletex**: This tool is employed for invoicing and expense handling.

These tools are aligned with Norwegian accounting regulations and support the company's financial management processes.
